<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2024-01-18 09:46:49--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  16.9MB/s    in 16s     

2024-01-18 09:47:06 (15.5 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [2]:
!pip install patool
import patoolib

patoolib.extract_archive('concrete_data_week4.zip')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 2.2 MB/s eta 0:00:00


INFO patool: Extracting concrete_data_week4.zip ...
INFO:patool:Extracting concrete_data_week4.zip ...
INFO patool: running /usr/bin/7z x -o./Unpack_7_q_g7q_ -- concrete_data_week4.zip
INFO:patool:running /usr/bin/7z x -o./Unpack_7_q_g7q_ -- concrete_data_week4.zip
INFO patool:     with input=''
INFO:patool:    with input=''
INFO patool: ... concrete_data_week4.zip extracted to `concrete_data_week4' (multiple files in root).
INFO:patool:... concrete_data_week4.zip extracted to `concrete_data_week4' (multiple files in root).


'concrete_data_week4'

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [14]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras.applications import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D

In [4]:
batch_size = 100

In [5]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [6]:
validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

In [7]:
train_generator = train_datagen.flow_from_directory(
    '/content/concrete_data_week4/concrete_data_week4/train',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 30001 images belonging to 2 classes.


In [8]:
validation_generator = validation_datagen.flow_from_directory(
    '/content/concrete_data_week4/concrete_data_week4/valid',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 9501 images belonging to 2 classes.


In [9]:
model = Sequential()
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
num_classes = 2
model.add(vgg16_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

58889256/58889256 [==============================] - 3s 0us/step


In [10]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

300/300 [==============================] - 664s 2s/step - loss: 1.0058 - accuracy: 0.9286 - val_loss: 0.0257 - val_accuracy: 0.9907


In [12]:
model.save('vgg16_classifier_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
model = Sequential()
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

num_classes = 2
model.add(resnet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

94765736/94765736 [==============================] - 5s 0us/step


In [16]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


model.save('resnet50_classifier_model.h5')

300/300 [==============================] - 616s 2s/step - loss: 0.0521 - accuracy: 0.9892 - val_loss: 0.3563 - val_accuracy: 0.9505


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model.
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [13]:
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

In [17]:
vgg16_model = load_model('/content/vgg16_classifier_model.h5')
resnet50_model = load_model('/content/resnet50_classifier_model.h5')

In [19]:
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_generator = test_datagen.flow_from_directory(
    '/content/concrete_data_week4/concrete_data_week4/test',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


vgg16_eval = vgg16_model.evaluate_generator(test_generator)
resnet50_eval = resnet50_model.evaluate_generator(test_generator)

Found 500 images belonging to 2 classes.


<ipython-input-19-b4d028354287>:14: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  vgg16_eval = vgg16_model.evaluate_generator(test_generator)
<ipython-input-19-b4d028354287>:15: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  resnet50_eval = resnet50_model.evaluate_generator(test_generator)


In [20]:
print("Performance of VGG16 Classifier:")
print("Loss:", vgg16_eval[0])
print("Accuracy:", vgg16_eval[1])


print("\nPerformance of ResNet50 Classifier:")
print("Loss:", resnet50_eval[0])
print("Accuracy:", resnet50_eval[1])

Performance of VGG16 Classifier:
Loss: 0.01446848176419735
Accuracy: 0.9959999918937683

Performance of ResNet50 Classifier:
Loss: 0.3951537311077118
Accuracy: 0.9459999799728394


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li>
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [21]:
predictions = resnet50_model.predict_generator(test_generator)

<ipython-input-21-682b908448c6>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = resnet50_model.predict_generator(test_generator)


In [22]:
for i in range(5):
    if predictions[i][0] > 0.5:
        print("Positive")
    else:
        print("Negative")

Positive
Positive
Positive
Positive
Positive


30001 IMAGES 

Check the learner's answer. The correct answer is 14,714,688

### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).